# Парсинг и анализ блога

### План
0. Подготовка
1. Сбор данных
2. Анализ данных
3. Визуализация 

In [5]:
from bs4 import BeautifulSoup
import requests
import re

import pandas as pd
import datetime
import time

import numpy as np
from matplotlib import pyplot as plt

##  1. Сбор данных

Объект изучения — блог Ильи Бирмана. Удобно, что движок блога делает отдельную [страницу](https://ilyabirman.ru/meanwhile/all/) со всеми постами.

In [6]:
webpage = requests.get("https://ilyabirman.ru/meanwhile/all/")

soup = BeautifulSoup(webpage.content, "html.parser")

Достать дату из поста оказалось сложнее, чем другие данные. В природе она выглядит так:
```html
<div class="e2-note-meta">
    <span class="e2-read-counter">
        <span class="e2-svgi">
            <svg>...</svg>
        </span>
        &nbsp;768
    </span>&nbsp;&nbsp;
    <span title="17 ноября 2018, 00:43, GMT+05:00">6 мес</span> ... 
</div>
```

…то есть внутри третьего `<span>` внутри `<div>`. Причем сама дата написана в `title=""`, а текстом показывается давность написания. 

Прочитал мануал по BeautifulSoup и RE и сначала пытался найти этот `span` как-то так: `soup.find_all("span", title=re.compile(".[\d]{4}.")`. Разумеется, ничего не вышло.

Потом ещё долго пытался достать нужный спан; в итоге добрался через родительский `div`, у которого определённ класс. Чтобы обратиться к третьему элементу внутри `div`, не сразу понял, что в этом случае надо к `soup` обращаться через `.contents`, а не `.children`.

Такой подход работал, пока я отлаживал подход на первой сотне постов. Когда выкатил парсер на более старые посты, у некоторых не оказалось счетчика просмотров. Из-за этого нумерация `span` с датой изменилась и парсер выдавал ошибки. Пришлось ещё раз прочитать мануал и прийти к более изящному решению — просто обратиться к дочернему `span` без класса у нужного `div`, без всяких номеров. 

В итоге достал всю `span` и через регулярные выражения вытащил отдельно день, месяц, год и время публикации поста. Месяц пришлось перевести в число с помощью списка месяцев. Всё сохранил в списке в формате объекта `datetime`. 

Получился длинный кусок кода, поэтому вытащил в отдельную функцию. Наверное, код, который вытаскивает остальные параметры поста, тоже надо было оформить в виде отдельных функций. Получилось бы аккуратнее.

In [7]:

def append_datetime_from_soup(source_soup, list_with_results):
    
    try:
        
        # first version of code: worked good only for posts with views counter; 
        # without counter, position of span with time and date is different 
        
        # find the <div>, get the <span> with timestamp (4th child)
        # span_datetime = str(source_soup.find("div", class_="e2-note-meta").contents[3])
            
        # updated verison doesn't care the position; it look for a span without class inside the right div
        span_datetime = str(source_soup.find("div", class_="e2-note-meta")\
                            .find("span", class_=""))
            
        # get string with date and time from <span> title
        date_str = re.search("\".+\"", span_datetime).group(0)

        # get day: one or two digits
        day = int(re.search("[\d]{1,2}", date_str).group(0))

        # get month as string
        month_str = str(re.search(" [\w^(\d)]+ ", date_str).group(0))
        month_str = month_str.strip()

        # convert string to integer
        months = ["января", "февраля", "марта", "апреля", "мая", "июня",
                  "июля", "августа", "сентября", "октября", "ноября", "декабря"]
        month_int = months.index(month_str) + 1

        # get year
        year = int(re.search("[\d]{4}", date_str).group(0))

        # get time
        time_ = re.search("[\d]{2}:[\d]{2}", date_str).group(0)
        hour = int(time_[0:2])
        minute = int(time_[3:5])

        # make a datetime object
        date_time = datetime.datetime(year, month_int, day, hour=hour, minute=minute)

        # append to the result
        list_with_results.append(date_time)
    
    except AttributeError:
        print("Can't find date in string {}"\
              .format(source_soup.find("div", class_="e2-note-meta")))
        list_with_results.append("0")
        
        
        
        
        

Удобно, что в Jupyter можно разделять куски кода: парсинг обычно занимает больше времени, чем обычные операции. Чтобы не ждать каждый раз, выделил его в отедльный кусок и запускал только при изменении кода.

Ещё было полезно органичить количество проходов для отладки, чтобы не парсить каждый раз все 4400+ заметок.

In [8]:
titles = []
views = []
comments  = []
tags = []
datetimes = []
images = []
words = []
links = []


# lists and counter for debugging purpose
bad_links = [] 
bad_links_meanwhile = []
parse_count = 0


# find all links
for link in soup.find_all("a"):
    post_tags = []
    parse_count += 1
    
    # drop not links (there are some 'None' object in scrapping results)
    if type(link.get('href')) == type("string"):
        
        # exclude blog engine settings links         
        if ("@ajax" in link.get('href'))\
        | ("/settings/" in link.get('href'))\
        | ("/tags/" in link.get('href'))\
        | (len(link.get("href")) <= 32) : # 32 is the length of "https://ilyabirman.ru/meanwhile/"
            continue
        
        # drop all except links for blogposts
        elif "ilyabirman.ru/meanwhile/" in link.get('href'):

            # get a link itself and parse it with with BeautifulSoup
            blog_page = requests.get(link.get('href'))
            blog_page_soup = BeautifulSoup(blog_page.content, "html.parser")
            #print(link.get("href")) # debugging
            
            # check if a post have a ciew counter (old posts have no views counter)
            if len(blog_page_soup.select(".e2-read-counter")) > 0:

                # get a span block with views count
                views_span = blog_page_soup.select(".e2-read-counter")[-1].get_text()

                # get a number from text block and format as integer, save to list of views count
                views.append(int(re.search("\d+", views_span).group(0)))

            else:
                views.append(1)
            
            # get comments count
            if blog_page_soup.find(id="e2-comments-count") != None:
                comments_span = blog_page_soup.find(id="e2-comments-count").get_text()
                comments.append(int(re.search("\d+", comments_span).group(0)))
            else:
                comments.append(0)
                
            # get tags for each post
            for tag in blog_page_soup.select(".e2-tag"):
                post_tags.append(tag.get_text())
            
            # list of posts' tags
            tags.append(post_tags)
            
            # get a blogpost title, save to list of titles
            titles.append(blog_page_soup.h1.get_text())
            
            # append date and time for each post to the list
            append_datetime_from_soup(blog_page_soup, datetimes)
            
            # get images count
            images.append(\
            len(blog_page_soup.find_all("div", class_="e2-text-picture-imgwrapper")))
            
            # post's length (words count)
            words.append(len(blog_page_soup.article.get_text().split()))
            
            # get link
            links.append(link.get('href'))
            
        else:
            
            # just checking the other links
            bad_links_meanwhile.append(link.get('href'))
    
    # checking, just in case
    elif type(link.get('href')) != None:
        bad_links.append(link.get('href'))
    
    else:
        print("Bad link: {}".format(link.get('href')))
    


In [9]:
# DEBUGGING and tweaking the links filter
# print(len(bad_links), len(bad_links_meanwhile))
# print(bad_links_meanwhile)
# print(len("https://ilyabirman.ru/meanwhile/"))

После прохода парсера на всем объёме блога проверил полность полученных данных (должно быть 4400+ записей).

Проверка показала только 2000 записей. Хорошо, что проверил :-)

Проблема оказалась в первом условии отбора подходящих ссылок. Отсеивал рабочие и прочие ссылки со страницы через проверку вхожения строки «meanwhile/all» в текст ссылки. Проверка простая и работала, но только на первой половине блога. На более старых постаъ частичка «/all» исчезла и записи не проходили проверку. 

Пришлось оставить только «meanwhile/all», но добавить перед этим ещё несколько отдельных условияй, чтобы отсеить рабочие ссылки.

Итоговые цифры: списки с данными постов одинаковой длины — 4483 записи.

In [10]:
# checking lengths of the lists with scrapping results
print(parse_count, len(views), len(images), len(words))

4595 4548 4548 4548


`dataframe` отказался принимать список списков с тегами для каждого поста, поэтому пришлось сделать список строк.

In [11]:
# convert list of lists with tags to list of strings (to add to a DataFrame properly)
tags_as_string = []

for item in tags:
    string = ''
    if type(item) == type(string): # if only 1 tag, add to the list it as is
        string = item
    else:
        for tag in item: # if more than one tag, iterate through
            string += tag + ", "
    tags_as_string.append(string[0:-2]) # add string of tags without last two chars: ", "

## 2. Анализ данных
После сбора данных, переходим к их анализу.

Чтобы работать с данными, собираем списки в словарь, а словарь — в `pandas dataframe`

In [12]:
dict = {"title": titles, 
       "datetime": datetimes,
        "views": views, 
        "comments": comments, 
       "length": words,
       "images": images,
        "tags": tags_as_string,
       "link": links} 

birman_frame = pd.DataFrame(data = dict)

birman_frame.drop_duplicates(inplace=True)
birman_frame.title = birman_frame.title.apply(lambda x: re.search("[^(\n|\r)]+", x).group(0))



Проверяем колонки и длину получившегося датафрейма. Минус два дупликата. Пустых значений нет.

In [13]:
birman_frame.info()

birman_frame.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4546 entries, 0 to 4547
Data columns (total 8 columns):
title       4546 non-null object
datetime    4546 non-null datetime64[ns]
views       4546 non-null int64
comments    4546 non-null int64
length      4546 non-null int64
images      4546 non-null int64
tags        4546 non-null object
link        4546 non-null object
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 319.6+ KB


title       0
datetime    0
views       0
comments    0
length      0
images      0
tags        0
link        0
dtype: int64

# запись в файл

In [14]:
birman_frame.to_csv("birman_new.csv", sep=";", index=False)

In [15]:
birman_from_file = pd.DataFrame()
birman_from_file = pd.read_csv("birman.csv", sep=";")

birman_from_file.info()
#birman_from_file.head()
#birman_from_file.isnull().sum()
#birman_from_file[birman_from_file.tags.isnull()].head()

birman_from_file.fillna("без тэга", inplace=True)

birman_from_file.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4487 entries, 0 to 4486
Data columns (total 8 columns):
title       4487 non-null object
datetime    4487 non-null object
views       4487 non-null int64
comments    4487 non-null int64
length      4487 non-null int64
images      4487 non-null int64
tags        4392 non-null object
link        4487 non-null object
dtypes: int64(4), object(4)
memory usage: 280.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4487 entries, 0 to 4486
Data columns (total 8 columns):
title       4487 non-null object
datetime    4487 non-null object
views       4487 non-null int64
comments    4487 non-null int64
length      4487 non-null int64
images      4487 non-null int64
tags        4487 non-null object
link        4487 non-null object
dtypes: int64(4), object(4)
memory usage: 280.5+ KB


Есть четыре числовых колонки, по которым можно сравнивать посты:
- просмотры
- комментарии
- длина поста
- количество картинок

Для начала самое простое: самые просматриваемые посты, самые комментируемые, длинные и где больше всего картинок.

In [16]:
# tell pandas not to truncate links in column
pd.options.display.max_colwidth = 100

In [17]:
birman_frame.sort_values(by='views', ascending=False).head(10)


,title,datetime,views,comments,length,images,tags,link
3160,О запятой после «С уважением»,2006-06-27 12:59:00,99381,10,174,0,русский язык,https://ilyabirman.ru/meanwhile/2006/06/27/2/
1719,Переплата по кредиту,2013-05-02 17:09:00,39417,0,278,0,"жизнь, общество, экономика",https://ilyabirman.ru/meanwhile/all/pereplata/
1827,Числа π и e,2012-12-28 13:16:00,15587,0,506,2,математика,https://ilyabirman.ru/meanwhile/all/pi-and-e/
1083,Война,2015-10-08 18:55:00,13810,0,627,0,"красная таблетка, общество",https://ilyabirman.ru/meanwhile/all/war/
1322,Почему люди платят налоги,2014-10-01 01:29:00,10453,194,1493,0,"красная таблетка, общество, философия, экономика",https://ilyabirman.ru/meanwhile/all/why-people-pay-taxes/
2241,О наращении окончаний числительных,2010-11-08 14:35:00,9823,19,354,0,"русский язык, студентам",https://ilyabirman.ru/meanwhile/all/o-naraschenii-okonchaniy-chislitelnyh/
1255,Почему нельзя перейти на доллары,2014-12-31 14:50:00,9403,0,267,0,"биткоин, общество, экономика",https://ilyabirman.ru/meanwhile/all/pochemu-nelzya-pereyti-na-dollary/
416,Рабочая неделя в Израиле,2018-01-27 14:05:00,7423,1,535,0,"из Тель-Авива, Израиль, Тель-Авив",https://ilyabirman.ru/meanwhile/all/shavua/
2403,Люголь,2009-09-09 11:58:00,7010,21,111,0,медицина,https://ilyabirman.ru/meanwhile/2009/09/09/1/
2110,Синхронное и асинхронное,2011-09-14 03:45:00,6818,38,87,0,"веб-разработка, русский язык",https://ilyabirman.ru/meanwhile/all/sync-async/


In [18]:
birman_frame.sort_values(by='comments', ascending=False).head(15)

,title,datetime,views,comments,length,images,tags,link
2490,Ремонетизация,2009-04-28 15:46:00,5,200,176,0,"реклама, этот сайт",https://ilyabirman.ru/meanwhile/2009/04/28/2/
2384,Бананотехнология,2009-11-06 01:22:00,97,199,220,0,"еда, жизнь",https://ilyabirman.ru/meanwhile/2009/11/06/1/
1322,Почему люди платят налоги,2014-10-01 01:29:00,10453,194,1493,0,"красная таблетка, общество, философия, экономика",https://ilyabirman.ru/meanwhile/all/why-people-pay-taxes/
2287,Опенсос,2010-06-04 00:48:00,29,174,442,0,"идиоты, опенсорс, софт",https://ilyabirman.ru/meanwhile/2010/06/04/1/
2863,Кто на чём,2007-10-08 12:44:00,5,150,88,0,браузеры,https://ilyabirman.ru/meanwhile/2007/10/08/1/
2133,Эгея,2011-07-05 21:07:00,84,123,136,1,"проекты, Эгея",https://ilyabirman.ru/meanwhile/all/aegea/
4126,Комментатор Саша,2004-02-06 10:42:00,38,116,279,0,"Елисейкин, идиоты, снукер",https://ilyabirman.ru/meanwhile/2004/02/06/2/
2561,"Переводим названия фирм, коллективов и продуктов",2009-01-21 04:35:00,221,114,197,0,"вопрос, русский язык",https://ilyabirman.ru/meanwhile/2009/01/21/1/
2261,Однажды в Сбербанке,2010-09-12 02:26:00,644,104,1253,0,"жизнь, переговоры, сервис",https://ilyabirman.ru/meanwhile/all/sberbank/
2138,Недовольство и возмущение,2011-06-30 19:52:00,55,92,37,0,"вопрос, я",https://ilyabirman.ru/meanwhile/all/discontent/


Самые длинные записи — конспекты книг и путеводитель по берлинскому клубу

In [19]:
birman_frame.sort_values(by='length', ascending=False).head()

,title,datetime,views,comments,length,images,tags,link
94,Книга Николая Никулина «Воспоминания о войне»,2019-05-05 11:22:00,2353,8,6026,1,книги,https://ilyabirman.ru/meanwhile/all/nikolay-nikulin-vospominaniya-o-voyne/
143,Книга Россера Ривза «Реальность в рекламе»,2019-01-14 12:33:00,1634,1,3750,1,"книги, реклама",https://ilyabirman.ru/meanwhile/all/reeves-advertizing-book/
133,Книга Чарльза Тарта «Практика внимательности в повседневной жизни». Часть 2,2019-01-30 00:55:00,1236,0,3563,0,"книги, медитация",https://ilyabirman.ru/meanwhile/all/tart-living-the-mindful-life-book-2/
170,Полный гид по клубу Бергхайн,2018-12-09 12:27:00,4120,6,3493,6,Бергхайн,https://ilyabirman.ru/meanwhile/all/berghain-guide/
135,Книга Чарльза Тарта «Практика внимательности в повседневной жизни». Часть 1,2019-01-28 14:45:00,1592,1,3198,0,"книги, медитация",https://ilyabirman.ru/meanwhile/all/tart-living-the-mindful-life-book-1/


Больше всего картинок оказалось в рассказе о проекте. Далее отчеты о телеграме и поездках

In [ ]:
birman_frame.sort_values(by='images', ascending=False).head(10)

,title,datetime,views,comments,length,images,tags,link
4,Блог,2019-08-24 10:04:00,1031,0,22,55,"видео, Школа стажёров, мир, Стокгольм, аудио по четвергам, техно, видеоблог, общество, подкаст, ...",https://ilyabirman.ru/meanwhile//
245,Процесс создания логотипа Драйвинг-тестов. Часть 1,2018-09-06 14:06:00,1377,3,1153,36,"портфолио, процесс",https://ilyabirman.ru/meanwhile/all/driving-tests-process-1/
381,Телеграм за неделю 5—11 февраля 2018,2018-02-26 11:44:00,824,0,1006,32,телеграм-канал,https://ilyabirman.ru/meanwhile/all/telegram-2018-feb-12/
380,Телеграм за неделю 12—18 февраля 2018,2018-02-28 23:44:00,831,1,1334,30,телеграм-канал,https://ilyabirman.ru/meanwhile/all/telegram-2018-feb-19/
231,Санкт-Петербург: Гранд-макет,2018-09-21 15:26:00,1591,2,262,29,"мир, музеи и выставки, Санкт-Петербург",https://ilyabirman.ru/meanwhile/all/saint-petersburg-grand-maket/
197,Тель-Авив: прогулка по Флорентину,2018-11-02 11:03:00,891,1,208,28,"мир, Тель-Авив",https://ilyabirman.ru/meanwhile/all/tel-aviv-2017-2018-florentin/
519,Телеграм за неделю 24—30 июля 2017,2017-11-01 12:47:00,17,1,1054,27,телеграм-канал,https://ilyabirman.ru/meanwhile/all/telegram-2017-jul-31/
196,Телеграм за неделю 4—10 сентября 2017,2018-11-05 16:18:00,971,2,695,26,телеграм-канал,https://ilyabirman.ru/meanwhile/all/telegram-2017-sep-11/
926,Музей БМВ в Мюнхене,2016-07-26 12:32:00,45,0,112,26,"автомобиль, Германия, музеи и выставки, фото",https://ilyabirman.ru/meanwhile/all/munich-bmw-museum/
766,Регистрация на рейс «Аэрофлота»,2017-02-08 01:37:00,3678,0,1464,24,"полёты, пользовательский интерфейс, студентам",https://ilyabirman.ru/meanwhile/all/aeroflot-registration/


In [ ]:
blog_page = requests.get("https://ilyabirman.ru/meanwhile/all/weekend-reading-167/")
blog_page_soup = BeautifulSoup(blog_page.content, "html.parser")


## 3. Визулизация данных



`DataFrame` умеет обращатсья с форматом `datetime` 

http://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#iterating-through-groups

особенно `strftime()`

Сначала сделал отображение в формате ГГГГ-НН вручную:
`birman_frame["datetime"].apply(lambda x: str(x. year) +"-"+ str("{:02d}".format(x.month))))` 

Потом почитал мануал и понял, что оно умеет само так делать:
`birman_frame.datetime.dt.strftime("%Y-%U")`


In [ ]:
"""
birman_frame["year"] = birman_frame.datetime.apply(lambda x: x.year)

#birman_frame.groupby('year').views.median()

months_comments = birman_frame.groupby(birman_frame["datetime"]\
                     .apply(lambda x: str(x. year) +"-"+ str("{:02d}".format(x.month))))\
                     .comments.sum()

#months_comments.plot()

months_views = birman_frame.groupby(birman_frame["datetime"]\
                     .apply(lambda x: str(x. year) +"-"+ str("{:02d}".format(x.month))))\
                     .views.sum()
#months_views.plot()

months_total = birman_frame.groupby(birman_frame["datetime"]\
                     .apply(lambda x: str(x.year) +"-"+ str("{:02d}".format(x.month))))\
                     .sum()

#plt.figure(figsize=(12,6))

# months_plot = months_total[["views", "comments", "length", "images"]].plot(kind="bar", subplots = True, figsize=(12,6))

#plt.hist(weeks_comments)
"""

# birman_frame.drop(columns=["year"], inplace=True)

'\nbirman_frame["year"] = birman_frame.datetime.apply(lambda x: x.year)\n\n#birman_frame.groupby(\'year\').views.median()\n\nmonths_comments = birman_frame.groupby(birman_frame["datetime"]                     .apply(lambda x: str(x. year) +"-"+ str("{:02d}".format(x.month))))                     .comments.sum()\n\n#months_comments.plot()\n\nmonths_views = birman_frame.groupby(birman_frame["datetime"]                     .apply(lambda x: str(x. year) +"-"+ str("{:02d}".format(x.month))))                     .views.sum()\n#months_views.plot()\n\nmonths_total = birman_frame.groupby(birman_frame["datetime"]                     .apply(lambda x: str(x.year) +"-"+ str("{:02d}".format(x.month))))                     .sum()\n\n#plt.figure(figsize=(12,6))\n\n# months_plot = months_total[["views", "comments", "length", "images"]].plot(kind="bar", subplots = True, figsize=(12,6))\n\n#plt.hist(weeks_comments)\n'

In [ ]:
by_weeks = birman_frame.groupby(birman_frame.datetime.dt.strftime("%Y-%U")).sum()
by_weeks["count"] = birman_frame.groupby(birman_frame.datetime.dt.strftime("%Y-%U")).views.count()

by_weeks.head()

,views,comments,length,images,count
datetime,,,,,
2002-06,59,0,370,2,3
2002-07,40,0,586,2,5
2002-08,8,0,269,2,1
2002-48,24,0,1286,1,2
2003-04,44,0,294,2,1


In [ ]:
by_months = birman_frame.groupby(birman_frame.datetime.dt.strftime("%Y-%m")).sum()
by_months["count"] = birman_frame.groupby(birman_frame.datetime.dt.strftime("%Y-%m")).views.count()

by_months.reset_index(inplace=True)

by_months.tail()



,datetime,views,comments,length,images,count
193,2019-04,11581,45,2334,39,14
194,2019-05,34495,81,12406,92,29
195,2019-06,27498,78,4960,84,23
196,2019-07,32300,79,4658,106,29
197,2019-08,13026,32,1231,131,14


In [ ]:
plot_one = by_weeks.plot(kind="bar", layout = (5,1), subplots = True, figsize=(12,6), title = "")

In [ ]:
plt.figure(figsize=(12,6))

plt.subplot(511)
plt.bar("index", "views", width=0.1, data=by_months, linewidth = 0.1)

plt.show

In [ ]:
by_months.plot.bar(y="views")


In [ ]:
by_months.plot(kind="bar", layout = (5,1), 
               subplots = True, figsize=(12,6), title = False)

In [ ]:
birman_frame.views

In [ ]:
views_std = birman_frame.views.std()
views_mean = birman_frame.views.mean()

#print(birman_frame.shape[0])

birman_frame[(birman_frame["views"] < (views_mean + 5*views_std)) &\
            (birman_frame["views"] > (views_mean - 5*views_std))].views.count()

views_outliers = birman_frame[(birman_frame["views"] > (views_mean + 5*views_std))]

views_outliers


In [ ]:
comments_std = birman_frame.comments.std()
comments_mean = birman_frame.comments.mean()

print(birman_frame.shape[0])

birman_frame[(birman_frame["comments"] < (comments_mean + 5*comments_std)) &\
            (birman_frame["comments"] > (comments_mean - 5*comments_std))].comments.count()

comments_outliers = birman_frame[(birman_frame["comments"] > (comments_mean + 5*comments_std))]

comments_outliers.sort_values("comments", ascending=False).head()


In [ ]:
birman_no_outliers = birman_frame[\
                    (birman_frame["comments"] < (comments_mean + 5*comments_std)) &\
                    (birman_frame["comments"] > (comments_mean - 5*comments_std)) &\
                    (birman_frame["views"] < (views_mean + 5*views_std)) &\
                    (birman_frame["views"] > (views_mean - 5*views_std))]

print(birman_frame.shape[0] - birman_no_outliers.shape[0])

In [ ]:
hours_to_views = birman_frame.groupby(birman_frame.datetime.dt.strftime('%H')).sum()

hours_to_views["hour"] = hours_to_views.index
hours_to_views["hour"] = hours_to_views["hour"].apply(lambda x: int(x))

hours_to_views.plot(x="hour", y="views",
                    kind="bar",
                    xticks = range(6, len(hours_to_views["hour"])+6),
                    figsize=(8,4))

In [ ]:
#birman_frame.length.size

fig = plt.figure(figsize=(12,6), frameon=False)

#fig.patch.set_visible(False)
ax = fig.add_axes([0, 0, 1, 1])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#ax.spines['left'].Spine(alpha=0.5)
#ax.axis('off')

plt.scatter("comments", "length", s="views",
            data = birman_no_outliers[birman_no_outliers["views"] > 50],
            marker = "o", alpha=0.1)
plt.ylabel("length")
plt.xlabel("comments")

plt.show()

In [ ]:
no_outliers_by_months = birman_no_outliers\
                        .groupby(birman_no_outliers.datetime.dt.strftime("%Y-%m"))\
                        .sum()

no_outliers_by_months.reset_index(inplace=True)

years = birman_no_outliers[birman_no_outliers.datetime.dt.strftime("%Y") > "2015"]\
    ["datetime"].sort_values(ascending=False).dt.strftime("%Y").unique()

years.sort()

print(len(years))


In [ ]:
count = 0

fig, axes = plt.subplots(nrows=len(years), ncols=1, figsize=(12,12))

plt.subplots_adjust(wspace = 0.05)

for year in years:
    birman_no_outliers\
        [birman_no_outliers.datetime.dt.strftime("%Y") == year]\
        ["length"].plot(kind="hist", bins=50,
                        ax=axes[count], sharey=True, sharex = True,
                        rot = 0, label = year, histtype = "stepfilled")
    plt.ylabel = year
    count+=1

#axes[0].set_title("views")
#axes[0].set_xticks(x_ticks)


#plt.show()

In [ ]:
for year in birman_no_outliers.datetime.dt.strftime("%Y").unique():
    print(year)